In [ ]:
import requests

mandatory_cities = [
    "Augsburg",
    "Ingolstadt",
    "Regensburg",
    "Landshut",
    "Straubing",
    "Passau",
    "Deggendorf",
    "Burghausen",
    "Weißenburg, Bayern",
]

optional_cities = [
    "Freising",
    "Moosburg a.d. Isar",
    "Vilsbiburg",
    "Landau a.d. Isar",
    "Mainburg",
    "Abensberg",
    "Schierling",
    "Schrobenhausen",
    "Erding",
    "Pfarrkirchen",
    "Osterhofen",
    "Eichstätt",
    "Neuburg a.d. Donau",
    "Kösching",
    "Pfaffenhofen a.d. Ilm",
]


HEADERS = {
    "User-Agent": "graphentheory/1.0"
}

def get_lat_long(city):
    response = requests.get(
        f"https://nominatim.openstreetmap.org/search",
        params={"city": city, "format": "json"},
        headers=HEADERS
    )
    response.raise_for_status()  # Raises an error if request failed
    data = response.json()
    if data:
        return {"lat": data[0]['lat'], "lon": data[0]['lon']}
    else:
        print(f"{city}: No results found")

cities_with_lat_lon = {}
for city in [*mandatory_cities, *optional_cities]:
    cities_with_lat_lon[city] = get_lat_long(city)

In [ ]:
import json

with open("city_coordinates.json", "w") as f:
    for v in cities_with_lat_lon.values():
        v["lat"] = float(v["lat"])
        v["lon"] = float(v["lon"])
    json.dump(cities_with_lat_lon, f, indent=4)

In [ ]:
cities = [*mandatory_cities, *optional_cities]

In [52]:
from calculate_distance import get_distance
import pandas as pd

min_dist = float("inf")
max_dist = 0

min_dist_cities = ()
max_dist_cities = ()

distance_between_cities = []

for city_1 in cities:
    for city_2 in cities:
        c1_lat_lon = cities_with_lat_lon[city_1]
        c2_lat_lon = cities_with_lat_lon[city_2]

        dist = get_distance(c1_lat_lon["lat"], c1_lat_lon["lon"], c2_lat_lon["lat"], c2_lat_lon["lon"])
        if dist > max_dist:
            max_dist_cities = (city_1, city_2)
            max_dist = dist
        if dist < min_dist:
            min_dist_cities = (city_1, city_2)
            min_dist = dist
        distance_between_cities.append({"from": city_1, "to": city_2, "distance": dist})

pd.DataFrame(distance_between_cities).to_csv("distances_between_cities.csv")

print(f"min dist {min_dist_cities} {min_dist}")
print(f"max dist {max_dist_cities} {max_dist}")

min dist ('Augsburg', 'Augsburg') 0.0
max dist ('Augsburg', 'Passau') 190.31931279878438


In [ ]:
https://router.project-osrm.org/route/v1/driving/LON1,LAT1;LON2,LAT2?overview=false
https://router.project-osrm.org/route/v1/driving/13.4609744,48.5748229;10.8979522,48.3690341?overview=false

In [ ]:
def get_driving_distance(lat_1, lon_1, lat_2, lon_2):
    response = requests.get(
        f"https://router.project-osrm.org/route/v1/driving/{lon_1},{lat_1};{lon_2},{lat_2}",
        params={"overview":"false"},
        headers=HEADERS
    ).json()
    distance = float("inf")
    for route in response["routes"]:
        distance = min(distance, route["distance"])
    return distance

#{'code': 'Ok', 'routes': [{'legs': [{'steps': [], 'weight': 9016.3, 'summary': '', 'duration': 9016.3, 'distance': 244460.6}], 'weight_name': 'routability', 'weight': 9016.3, 'duration': 9016.3, 'distance': 244460.6}], 'waypoints': [{'hint': 'GeIDhv___38PAAAAFQAAAHIAAAAAAAAAgqLbQHlLJUBFUUlCAAAAAA8AAAAVAAAAcgAAAAAAAADpEQEA4GXNAGk05QLuZc0AZzHlAgQAnwUAAAAA', 'location': [13.46096, 48.575593], 'name': 'Roßtränke', 'distance': 85.63138398}, {'hint': 'Ccd4gHruxYEVAAAAAAAAACEBAAAAAAAAxJoXQQAAAADQ4PBCAAAAABUAAAAAAAAAIQEAAAAAAADpEQEA8kqmAO4O4gIgSqYAig3iAhIA_woAAAAAee7Fgf___38AAAAAAgAAAAAAAAAyAAAAAAAAAN3Gvz8AAAAA8nkGQgAAAAACAAAAAAAAADIAAADpEQEA8kqmAO4O4gIgSqYAig3iAgAAHwUAAAAA', 'location': [10.898162, 48.36939], 'name': 'Karolinenstraße / Am Perlachberg', 'distance': 42.53366056}]}

print(get_driving_distance(48.5748229, 13.4609744,48.3690341, 10.8979522))

In [49]:
driving_distances = {city: {} for city in cities}

df = pd.read_csv("driving_distances_between_cities.csv")
for _, row in df.iterrows():
    driving_distances[row["from"]][row["to"]] = row["distance"]

def save():
    result_driving_distances = []
    for from_city, to_cities in driving_distances.items():
        for to_city, distance in to_cities.items():
            result_driving_distances.append({"from": from_city, "to": to_city, "distance": distance})

    pd.DataFrame(result_driving_distances).to_csv("driving_distances_between_cities.csv")

i=0
for city_1 in cities:
    for city_2 in cities:
        if city_1 == city_2:
            driving_distances[city_1][city_2] = 0
            save()
            continue
        if city_2 in driving_distances[city_1]:
            continue
        c1_lat_lon = cities_with_lat_lon[city_1]
        c2_lat_lon = cities_with_lat_lon[city_2]
        i+=1
        print(f"from{city_1} {city_2}")
        distance = get_driving_distance(c1_lat_lon["lat"], c1_lat_lon["lon"], c2_lat_lon["lat"], c2_lat_lon["lon"])
        driving_distances[city_1][city_2] = distance
        driving_distances[city_2][city_1] = distance
        save()

In [51]:
from calculate_distance import get_distance
import pandas as pd

min_dist = float("inf")
max_dist = 0

min_dist_cities = ()
max_dist_cities = ()

for city_1 in cities:
    for city_2 in cities:
        if city_1 == city_2:
            continue
        dist = driving_distances[city_1][city_2]
        if dist > max_dist:
            max_dist_cities = (city_1, city_2)
            max_dist = dist
        if dist < min_dist:
            min_dist_cities = (city_1, city_2)
            min_dist = dist
        
print(f"min dist {min_dist_cities} {min_dist}")
print(f"max dist {max_dist_cities} {max_dist}")

min dist ('Ingolstadt', 'Kösching') 9626.4
max dist ('Passau', 'Weißenburg, Bayern') 261615.8
